In [1]:
import numpy as np
import time
from numba import jit

In [2]:
np.random.seed(0)
A = np.random.rand(100000,3)*500

x_bins = np.linspace(0, 500, num=50)

@jit
def count_3D_simple(A, x_bins, y_bins, z_bins):
    
    hist = np.zeros((len(x_bins)-1, len(y_bins)-1, len(z_bins)-1))
    
    for i in range(len(x_bins)-1):
        for j in range(len(y_bins)-1):
            for k in range(len(z_bins)-1):
                hist[i,j,k] = np.sum((A[:,0] >= x_bins[i]) * (A[:,0] < x_bins[i+1]) * 
                                     (A[:,1] >= y_bins[j]) * (A[:,1] < y_bins[j+1]) * 
                                     (A[:,2] >= z_bins[k]) * (A[:,2] < z_bins[k+1]))
                
    return hist

@jit
def count_3D_optimized(A, x_bins, y_bins, z_bins):

    hist = np.zeros((len(x_bins)-1, len(y_bins)-1, len(z_bins)-1))

    for i in range(len(x_bins)-1):
        a = A[:,0] >= x_bins[i]
        np.less(A[:,0], x_bins[i+1], out=a, where=a)
        
        for j in range(len(y_bins)-1):
            b = a.copy()
            np.greater_equal(A[:,1], y_bins[j], out=b, where=b)
            np.less(A[:,1], y_bins[j+1], out=b, where=b)
            
            for k in range(len(z_bins)-1):
                c = b.copy()
                np.greater_equal(A[:,2], z_bins[k], out=c, where=c)
                np.less(A[:,2], z_bins[k+1], out=c, where=c)
                
                hist[i,j,k] = np.sum(c)

    return hist

In [3]:
t0 = time.time()

hist1 = count_3D_simple(A, x_bins, x_bins, x_bins)

print('Seconds to complete:', time.time()-t0)

Seconds to complete: 42.26777148246765


In [4]:
t0 = time.time()

hist2 = count_3D_optimized(A, x_bins, x_bins, x_bins)

print('Seconds to complete:', time.time()-t0)

Seconds to complete: 21.16849994659424


In [5]:
np.allclose(hist1,hist2)

True

In [6]:
hist1

array([[[0., 4., 1., ..., 0., 0., 2.],
        [3., 2., 1., ..., 2., 1., 1.],
        [1., 4., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 4., ..., 4., 1., 2.],
        [1., 0., 0., ..., 1., 1., 2.],
        [1., 1., 2., ..., 1., 1., 0.]],

       [[1., 0., 5., ..., 3., 1., 1.],
        [0., 3., 0., ..., 3., 2., 0.],
        [1., 1., 2., ..., 1., 0., 0.],
        ...,
        [1., 0., 0., ..., 1., 1., 1.],
        [0., 0., 1., ..., 1., 2., 0.],
        [2., 0., 1., ..., 1., 1., 0.]],

       [[2., 1., 2., ..., 0., 0., 1.],
        [1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 0., 0.],
        ...,
        [1., 1., 0., ..., 0., 1., 2.],
        [1., 0., 1., ..., 2., 1., 2.],
        [0., 1., 0., ..., 1., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 1., 0., 1.],
        [0., 2., 1., ..., 1., 0., 1.],
        [1., 2., 0., ..., 0., 1., 2.],
        ...,
        [0., 0., 1., ..., 0., 1., 3.],
        [0., 0., 0., ..., 3., 2., 1.],
        [0., 1., 1., ..., 0., 1.